<a href="https://colab.research.google.com/github/kshitijsinghh/CustomerSatisfactionAndEfficiencyMetrics/blob/main/CustomerSatisfactionAndEfficiencyMetrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 5.3 MB/s 


In [2]:
import csv
import pandas as pd
import numpy as np
import datetime as dt
import xlsxwriter

df= pd.read_csv('data.csv')  

df['ChatStartTime']= pd.to_datetime(df['ChatStartTime'])  #CONVERTING TIME COLUMNS INTO DATETIME FORMAT
df['ChatEndTime']=pd.to_datetime(df['ChatEndTime'])
df['is_bus_hr']=0        # HERE I HAVE MADE ANOTHER TABLE THAT MARK WHETHER THE CHATS ARE IN BUSINESS HOURS OR NOT
j=0
for i in df['ChatStartTime']:
   if(i.hour>9 ):
     if(i.hour<17 ):

      
      df.at[j, 'is_bus_hr'] = 1
   else:
     
     

     df.at[j, 'is_bus_hr']=0
   j=j+1

Number_of_Chats_weekly= df.groupby(pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True)).size().reset_index(name = 'Number_of_Chats_weekly') #GROUPED THE DATA ON WEEKLY BASIS AND COUNTED NUMBER OF ROWS AS EACH ROW IS NEW CHAT
#print(Number_of_Chats_weekly)

Unique_users = df.drop_duplicates("UserId", keep='first')  #CREATED A DATAFRAME OF ALL UNIQUE USERS SO THAT WHEN WE COUNT THE NUMBER OF ROWS AND GROUP THEM ON WEEKLY BASIS 
Number_of_Unique_Users_weekly= Unique_users.groupby(pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True)).size().reset_index(name = 'Number_of_Unique_Users_weekly')
#print(Number_of_Unique_Users_weekly)
closed_by_bots= df[(df['ClosedBy']=='System') | (df['ClosedBy']=='Bot') ] #CREATED A DATAFRAME OF ALL THE CHATS CLOSED ONLY BY BOTS AND SYSTEM USNG BOOLEAN MASKING
closed_by_bots_weekly= closed_by_bots.groupby(pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True)).size().reset_index(name = 'Closed_by_bots_weekly')  #GROUPED THERE COUNTS ON WEEKLY BASIS
#print(closed_by_bots_weekly)
Merged1 = pd.merge(Number_of_Chats_weekly,Number_of_Unique_Users_weekly, how='inner', on='ChatStartTime') #MERGING 2 DATAFRAMES A TIME TO MAKE FINAL RESULT DATAFRAME AS ON MERING ALL AT ONCE WAS GIVING RUNTIME PROBLEM
Merged2= pd.merge(Merged1,closed_by_bots_weekly, how='inner', on='ChatStartTime') #MERGING DATAFRAMES
Agents = (df['ClosedBy']!='Bot') & (df['ClosedBy']!='System')
Closed_By_Agents = df[Agents] #CREATED DATAFRAME OF ALL THE CHATS CLOSED BY AGENTS
closed_by_Agents_weekly= Closed_By_Agents.groupby(pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True)).size().reset_index(name = 'Closed_by_Agents_weekly') #GROUPED THE CHATS CLOSED BY AGENTS ON WEEKLY BASIS
Merged= pd.merge(Merged2,closed_by_Agents_weekly, how='inner', on='ChatStartTime')

Merged['Bot_Deflection %']= (Merged['Closed_by_bots_weekly']/Merged['Number_of_Chats_weekly'])*100 #CALCULATED  BOT DEFLECTION % LIKE HOW MANY CHATS WERE CLOSED BY BOS OUT OF TOTAL CHATS
Merged.rename(columns={'ChatStartTime':'WeekWiseDates'},inplace=True)
Merged['WeekWiseDates']= list(map(lambda x: x.date() , Merged['WeekWiseDates'])) #OUT OF TIMESTAMP TAKING ONLY THE DATES

Merged.to_csv("ChatMetricsReport.csv",index=False,na_rep='NULL')  #TILL HERE WE ARE DONE WITH CHAT METRICS SO MAKING A SEPERATE CSV FOR THAT AND IN THE END WILL FINAL XLSX FILE
print(Merged.head())



# AGENT MTERICS SECTION


mask = (df['ClosedBy']!='Bot') & (df['ClosedBy']!='System') 
Closed_By_Agents = df[mask]
Closed_By_Agents.reset_index(inplace=True) #CREATED A DATAFRAME OF CHATS CLOSED BY AGENTS ONLY AND RESET ITS INDEX AS LATER WE'LL DO MULI-INDEXING ON IT

Chats_Resolved_By_Each_Agent = Closed_By_Agents.groupby([pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True),'ClosedBy']).size().reset_index(name='Chats_Resolved_By_Each_Agent') #GROUPED HOW MANY CHATS WERE CLOSED BASED ON WEEKS AND AGENTS NAMES
#print(Chats_Resolved_By_Each_Agent) 




aa= pd.to_timedelta(Closed_By_Agents['AgentFirstResponseTime'])


avg_agent_time = list(map(lambda x: x.total_seconds() , aa)) #CONVERTED TOTAL FIRST RESPONSE TIME IN SECONDS FORMAT SO THAT TAKING MEAN WOULD BE EASY


ab = pd.Series(avg_agent_time)
Closed_By_Agents['AgentFirstResponseTimeInSeconds']=ab 
#print(Closed_By_Agents['AgentFirstResponseTimeInSeconds'])
Avg_First_Response_Time = Closed_By_Agents.groupby([pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True),'ClosedBy']).agg({"AgentFirstResponseTimeInSeconds":np.nanmean}).rename(columns={'AgentFirstResponseTimeInSeconds': 'MeanAgentFirstResponseTimeInSeconds'})

#print(Avg_First_Response_Time)


Closed_By_Agents['Diff']= Closed_By_Agents['ChatEndTime']-Closed_By_Agents['ChatStartTime'] #GOT THE DIFFERENCE BETWEEN START AND END TIME 
diff_sec = list(map(lambda x: x.total_seconds() , Closed_By_Agents['Diff'])) #CONVERTED THAT DIFFERENCE INTO TOTAL SECONDS SO THAT WE CAN EASILY TAKE MEAN
# for i in Closed_By_Agents['Diff']:
#   diff_sec.append(i.total_seconds())
Closed_By_Agents['Diffsec']= pd.DataFrame(diff_sec)
test = Closed_By_Agents[['ChatStartTime','ChatEndTime','Diff','Diffsec']] #BY THIS YOU CAN CHECK WHETHER ABOVE STEPS ARE RIGHT OR NOT

Agent_Chat_Resolution_Time = Closed_By_Agents.groupby([pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True),'ClosedBy']).agg({"Diffsec":np.nanmean,"AgentFirstResponseTimeInSeconds":np.nanmean,"CsatScore":np.nanmean}).rename(columns={'Diffsec': 'Agent_Avg_Chat_Resolution_Time','AgentFirstResponseTimeInSeconds':'AgentFirstResponseTimeInSeconds',"CsatScore":'MeanCsatScore'})

#GROUPED AVERAGE CHAT RESOLUTION TIME GROUPED BY WEEKS AND AGENTS NAME

#print(Agent_Chat_Resolution_Time)

# merge1= pd.merge(Avg_First_Response_Time,Agent_Chat_Resolution_Time, how='outer', on='ChatStartTime')
# print(merge1)
Agent_CSAT_Score = Closed_By_Agents.groupby([pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True),'ClosedBy']).agg({"CsatScore":np.nanmean}).rename(columns={'CsatScore': 'MeanCsatScore'})

#CALCULATED THE MEAN CSAT SCORE GROUPED BY AGENTS NAME AND WEEKS
#print(Agent_CSAT_Score)

Contacted_in_businesHr = Closed_By_Agents[Closed_By_Agents['is_bus_hr']==1] #CREATED THE DATAFRAME OF ALL THE CHATS IN BUSINESS HOURS
Agent_CSAT_Score_in_BHr = Contacted_in_businesHr.groupby([pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True),'ClosedBy']).agg({"CsatScore":np.nanmean}).rename(columns={'CsatScore': 'MeanCsatScoreInBusinessHr'})
#OUT OF DATAFRAME CREATED GROUPED THE MEAN CSAT SCORE BY AGENTS NAME AND WEEKS

#print(Agent_CSAT_Score_in_BHr)
NotContacted_in_businesHr = Closed_By_Agents[Closed_By_Agents['is_bus_hr']==0]
Agent_CSAT_Score_not_in_BHr = NotContacted_in_businesHr.groupby([pd.Grouper(key='ChatStartTime', axis=0, 
                      freq='1W', sort=True),'ClosedBy']).agg({"CsatScore":np.nanmean}).rename(columns={'CsatScore':'MeanCsatScoreNotInBHr'})

#SIMILARLY CREATED THE DATAFRAME OF ALL THE CHATS IN NON BUSINESS HOURS AND GROUPED THE MEAN CSAT SCORE IN THAT TIME FRAME BY AGENTS NAME AND WEEKS
merged1 =  pd.merge(Agent_CSAT_Score_not_in_BHr,Agent_CSAT_Score_in_BHr, how='outer', on=['ChatStartTime','ClosedBy'])
#print(merged1)
merged2 = pd.merge(merged1,Agent_Chat_Resolution_Time, how='outer', on=['ChatStartTime','ClosedBy'])

# print(merged2)
merged_final = pd.merge(merged2,Chats_Resolved_By_Each_Agent, how='outer', on=['ChatStartTime','ClosedBy'])
merged_final.rename(columns={'ChatStartTime':'WeekWiseDates'},inplace=True)
merged_final['WeekWiseDates']= list(map(lambda x: x.date() , merged_final['WeekWiseDates']))
merged_final.to_csv("AgentMetricsReport.csv",index=True) #AGENT METRICS PART IS COMPLETE , SO CREATED ITS SEPERATE CSV AS WELL
print(merged_final.head())


#BONUS METRICS SECTION

mask = (df['ClosedBy']!='Bot') & (df['ClosedBy']!='System') 
df = df[mask]

df['AgentAssignmentTimestamp']=pd.to_datetime(df['AgentAssignmentTimestamp'], errors='coerce') #CONVERTED AgentAssignmentTimestamp IN DATE TIME FORMAT
df['CumulativeTime']=df['ChatEndTime']-df['ChatStartTime'] # CUMULATIVE TME
df['ActiveTime']=df['ChatEndTime']-df['AgentAssignmentTimestamp'] #ACTIVE TIME
ActiveTimeInSec = list(map(lambda x: x.total_seconds() , df['ActiveTime'])) #CONVETED IT TO TOTAL SECOND AS IT WOULD BE EASY TO CALCULATE MEAN
df['ActiveTimeInSec']= ActiveTimeInSec
df[['ActiveTime','ActiveTimeInSec']]
CumulativeTimeInSec = list(map(lambda x: x.total_seconds() , df['CumulativeTime'])) #CONVETED IT TO TOTAL SECOND AS IT WOULD BE EASY TO CALCULATE MEAN
df['CumulativeTimeInSec']= CumulativeTimeInSec
Bonus_Metrics = df.groupby('ClosedBy').agg({'CumulativeTimeInSec':np.nanmean,'ActiveTimeInSec':np.nanmean}).rename(columns={'CumulativeTimeInSec': 'MeanCumulativeTimeInSec','ActiveTimeInSec':'MeanActiveTimeInSec'})
# GROUPED THE MEAN CUMULATIVE TIME AND ACTIVE TIME BY AGENTS NAME
print(Bonus_Metrics.head())


writer = pd.ExcelWriter('Report.xlsx', engine='xlsxwriter')

#store your dataframes in a  dict, where the key is the sheet name you want
frames = {'AGENT_MERICS': merged_final, 'CHAT_METRICS': Merged, 'BONUS_METRCS':Bonus_Metrics }

#now loop thru and put each on a specific sheet
for sheet, frame in  frames.items(): # .use .items for python 3.X
    frame.to_excel(writer, sheet_name = sheet)

#critical last step
writer.save()
#HERE YOU'LL GET XLSX REPORT ALL THREE TABS IN ON SHEET



  WeekWiseDates  ...  Bot_Deflection %
0    2021-01-03  ...         65.326633
1    2021-01-10  ...               NaN
2    2021-01-17  ...               NaN
3    2021-01-24  ...               NaN
4    2021-01-31  ...        100.000000

[5 rows x 6 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

  WeekWiseDates  ClosedBy  ...  MeanCsatScore  Chats_Resolved_By_Each_Agent
0    2021-01-03      Adaa  ...       0.833333                             6
1    2021-01-03     Akash  ...       1.666667                             3
2    2021-01-03    Anisha  ...       3.333333                             3
3    2021-01-03     Azaan  ...       0.000000                             4
4    2021-01-03  Dalphine  ...       1.000000                             4

[5 rows x 8 columns]
          MeanCumulativeTimeInSec  MeanActiveTimeInSec
ClosedBy                                              
Abbas                  853.315789           727.578947
Adaa                  1356.685714          1249.557143
Adil                  1019.125000           792.093750
Ajay                   910.240741           777.888889
Akash                  600.440000           472.360000
